In [24]:
from os import path
if not path.exists('utils.py'): # If running on colab: the first time download and unzip additional files
    !wget https://biolab.csr.unibo.it/samples/fr/files.zip
    !unzip files.zip
    
    
import utils  # Run utils.py for helper functions
import math
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from utils import *
from ipywidgets import interact

In [ ]:
# Read Fingerprints in the fingerprint subdirectory
def read_fingerprints():

    directoryPath = './fingerprint-/DB1_B/'
    fingerprints = []
    resizedImages = []

    for filename in os.listdir(directoryPath):
        
        filepath = os.path.join(directoryPath, filename)
        image = cv.imread(filepath)

        if fingerprints is not None:

            fingerprints.append(image)

        else:
            print("Failed to Read image of %s\n" % filename)

    return fingerprints